<h1 style="font-family:verdana;"> <center>RetinaNet🎖 </center> </h1>

<h3><center style="color:#159364; font-family:cursive;">Validation and Submission Notebook</center></h3>


#### 1. Check out Training [Notebook](https://www.kaggle.com/akhileshdkapse/vinbigdata-retinanet-detection-training)


<h1 style="font-family:verdana;"> <center>Offline RetinaNet installation</center> </h1>

#### 2. Check out for .WHL processing [Notebook](https://www.kaggle.com/akhileshdkapse/creating-whl-file-retinanet)

In [ ]:
# Create 'RetinaNet' dir for zip extraction
%mkdir RetinaNet

In [ ]:
%%capture
!unzip ../input/vinbigdata-zip-data/Keras-Retinanet.zip -d ./RetinaNet

In [ ]:
%%capture

%cd RetinaNet/
!pip install ../../input/creating-whl-file-retinanet/whlfiles/keras_resnet-0.2.0-py2.py3-none-any.whl

In [ ]:
import pandas as pd
import numpy as np
import os
import time
import matplotlib.pyplot as plt
import cv2


from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr ,preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

path= '../../input/retinanet-50-th-epoch-model/resnet50_csv_50.h5'

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
# Inference model
model = models.load_model(path, backbone_name='resnet101')
model = models.convert_model(model)

<h1 style="font-family:verdana;"> <center>Model Validation</center> </h1>

In [ ]:
df_valid= pd.read_csv('annotations_test.csv', header=None)
df_valid.head()

In [ ]:
n_classes= pd.read_csv('classes.csv', header=None)

classes= {}
classes_r= {}
for name, roll in zip(n_classes[0], n_classes[1]):
    classes[name]= roll
    classes_r[roll]=name

In [ ]:
uniq= df_valid[0].unique().shape[0]
classes

In [ ]:
def predict(path, th=0.3, draw=True, show_time=True):
    
    # load image
    image = read_image_bgr(path)
    # copy to draw on
    image2 = image.copy()
    #image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)
    # preprocess image for network
    image = preprocess_image(image)
    image, scale = resize_image(image)
    
    # process image
    start = time.time()
    boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
    if show_time:
        print("processing time: ", time.time() - start)
    # correct for image scale
    boxes /= scale
    
    if draw:
        for box, score, label in zip(boxes[0], scores[0], labels[0]):
        # scores are sorted 
            if score < th:
                break
            color = label_color(label)
            b = box.astype(int)
            draw_box(image2, box, color=color)
            draw_caption(image2, box, classes_r[label])
        return image2
    else:
        return boxes[0], scores[0], labels[0]

In [ ]:
def grd_vs_pred(i, th= 0.3):
    path= df_valid[0].unique()[i]
    grd_img= read_image_bgr(path)

    for r, row in df_valid[df_valid[0]==df_valid[0][i]].iterrows():
        color= label_color(classes[row[5]])
        grd_img= cv2.rectangle(np.array(grd_img), (row[1], row[2]), (row[3], row[4]), color, 2)
        draw_caption(grd_img, row[1:5].values, row[5])
    pred_img= predict(path, th=th)
    
    return grd_img, pred_img

In [ ]:
# f, ax= plt.subplots(2, 4, figsize=(24, 10))

# for i in range(4):
#     np.random.seed(i*49)
#     grd, pred= grd_vs_pred(np.random.randint(uniq), th= 0.4)
#     ax[0][i%4].imshow(grd); ax[1][i%4].imshow(pred)
#     ax[0][i%4].set_title('Grd Truth')
#     ax[1][i%4].set_title('Predicted')
# plt.show()

<h1 style="font-family:verdana;"> <center>Result Submission</center> </h1>


In [ ]:
%cd ..

In [ ]:
# https://www.kaggle.com/awsaf49
class_2= pd.read_csv('../input/vinbigdata-2class-prediction/2-cls test pred.csv')
class_2.head()

In [ ]:
sub_512=pd.read_csv('../input/vinbigdata-512-image-dataset/vinbigdata/test.csv')
sub_ex=pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv')
sub_ex.head()

In [ ]:
sub=pd.DataFrame.merge(sub_ex,sub_512,on='image_id')
sub['extra']= sub.image_id +'--'+ sub.width.astype(str) +'--'+ sub.height.astype(str)
sub.head()

In [ ]:
def perdict_string(label_id, th=0.5):
    string= ''
    label_id, w, h= label_id.split('--')
    xfac= int(w)/512
    yfac= int(h)/512
    path='../input/vinbigdata-512-image-dataset/vinbigdata/test/'
    path= path + label_id + '.png'
    boxes, score, label= predict(path, th=0.2, draw=False, show_time=False)
    
    for l, s, b in zip(label, score, boxes):
        if s <th:
            break
        string+= '{} {:.2f} {} {} {} {} '.format(l ,s , int(b[0]*xfac), int(b[1]*yfac),
                                                 int((b[2])*xfac), int((b[3])*yfac))
        
    return string if len(string) else "14 1 0 0 1 1"

In [ ]:
sub['PredictionString']= sub.extra.apply(perdict_string)
sub= sub[['image_id', 'PredictionString']]
sub.head()

In [ ]:
pred = pd.merge(sub, class_2, on = 'image_id', how = 'left')
pred.head()

In [ ]:
def filter_2cls(row, thr=0.1):
    if row['target']<thr:
        row['PredictionString'] = '14 1 0 0 1 1'
    return row

In [ ]:
sub2 = pred.apply(filter_2cls, axis=1)
sub2= sub2[['image_id', 'PredictionString']]
sub2.head()

In [ ]:
sub.to_csv('/kaggle/working/submission.csv',index = False)



## 🌄 Thanks for Reading

![](https://i.gifer.com/7ImI.gif)



<div class="alert alert-block alert-info" style="font-size:20px; font-family:verdana;">
 <a target="_blank" style="color:orange;">Do UPVOTE for more Motivation🤞</a>
</div>



<hr><hr><hr>

<hr>